In [1]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
L = np.pi*2 * 2
T = 50

a = 1e-1
dx = 5e-2
dt = dx**2 / (6*a)

x = np.arange(0, L+dx, dx)
t = np.arange(0, T+dt, dt)

boundary_x = [0, -1]
Nt, Nx = len(t), len(x)

def u_0(x):
    "Initial Condition"
    return np.cos(x)

def u_b(t):
    "Boundary Conidtion for function"
    return 1

def du_b(t):
    "Boundary Conidtion for derivative"
    return 0

u = np.zeros((Nt, Nx))

BC = "Derichlet"

# if BC == "Derichlet":
#     for i, ti in enumerate(t):
#         u[1:, [0, -1]] = u_b(t)
    
def Du(ti, ui):
    du = np.zeros_like(ui)
    du[1:-1] = (ui[2:] - ui[:-2])/(2*dx)
    if BC == "Derichlet":
        du[0]  = (ui[1]  - u_b(ti))/(dx)
        du[-1] = (u_b(ti) - ui[-2])/(dx)
    elif BC == "Neumann":
        du[0]  = du_b(ti)
        du[-1] = du_b(ti)
    return du

def D2u(ti, ui):
    d2u       = np.zeros_like(ui)
    d2u[1:-1] = (ui[2:] - 2*ui[1:-1] + ui[:-2])/(dx**2)
    if BC == "Derichlet":
        d2u[0]  = (ui[2]  - 2*ui[1]  + u_b(ti))/(dx**2)
        d2u[-1] = (ui[-3] - 2*ui[-2] + u_b(ti))/(dx**2)
    elif BC == "Neumann":
        d2u[0]  = (ui[2]  - ui[1]  - dx*du_b(ti))/(dx**2)
        d2u[-1] = (ui[-3] - ui[-2] - dx*du_b(ti))/(dx**2)
    return d2u

def PDE(ti, ui):
    dudt = a*D2u(ti, ui)
    return dudt

for i, ti in enumerate(t):
    if i == 0:
        u[0, :] = u_0(x)
    else:
        ui = u[i-1, :]
        k1 = dt*PDE(ti, ui)
        k2 = dt*PDE(ti+0.5*dt, ui+0.5*k1)
        k3 = dt*PDE(ti+0.5*dt, ui+0.5*k2)
        k4 = dt*PDE(ti+dt, ui+k3)
        u[i, :] = ui + 1/6 * (k1 + 2*k2 + 2*k3 + k4)


In [3]:
def save_gif_PIL(outfile, files, fps=5, loop=0):
    "Helper function for saving GIFs"
    imgs = [Image.open(file) for file in files]
    imgs[0].save(fp=outfile, format='GIF', append_images=imgs[1:], save_all=True, duration=int(1000/fps), loop=loop)

def make_plot(save=False, file_name=f"PDE_solution.png"):
    "Helper function for plotting"
    fig, ax = plt.subplots()
    ax.plot(x, u[i, :])
    ax.set_xlim(0, L)
    ax.set_ylim(-1.05, 1.05)
    ax.grid()
    ax.set_title(f"Time = {t[i]:.3f}s")
    if save:
        fig.savefig(file_name, bbox_inches="tight", pad_inches=0.1, dpi=100, facecolor="white")

In [4]:
Nt//10

5000

In [5]:
files = []
for i in range(Nt):
    if i%100 == 0:
        file = f"plots/PDE_sol_{i:08d}.png"
        make_plot(save=True, file_name=file)
        files.append(file)
        plt.close("all")
save_gif_PIL("PDE_solution.gif", files, fps=20, loop=0)